Physics of flow

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

# Set up a simple power system as a graph
G = nx.DiGraph()

# Add nodes (buses) with voltage levels
buses = {
    "Gen": {"voltage": 100, "type": "generator"},
    "Load1": {"voltage": 95, "type": "load"},
    "Load2": {"voltage": 92, "type": "load"},
    "Substation": {"voltage": 98, "type": "substation"},
}
for node, attrs in buses.items():
    G.add_node(node, **attrs)

# Add edges (transmission lines) with resistance and reactance
# Note: In real AC systems, X is often > R for transmission lines.
lines = [
    ("Gen", "Substation", {"resistance": 0.1, "reactance": 0.5, "rating": 15}), # Lower R, higher X, add rating (e.g., MVA)
    ("Substation", "Load1", {"resistance": 0.2, "reactance": 1.0, "rating": 10}),
    ("Substation", "Load2", {"resistance": 0.3, "reactance": 1.5, "rating": 10}),
]
for u, v, attrs in lines:
    G.add_edge(u, v, **attrs)

# Calculate voltage drops using Ohm's Law (V_drop = I * R) - OBSOLETE for AC representation
# for u, v, data in G.edges(data=True):
#     data[\"voltage_drop\"] = data[\"current\"] * data[\"resistance\"]
# \n",

# Position nodes for visualization
pos = {
    "Gen": (0, 1),
    "Substation": (1, 1),
    "Load1": (2, 0.5),
    "Load2": (2, 1.5),
}

# Draw the graph
plt.figure(figsize=(10, 6))
nx.draw_networkx_nodes(
    G, pos, node_color="lightblue", node_size=2000, edgecolors="black"
)
nx.draw_networkx_labels(G, pos, font_size=12, font_weight="bold")

# Draw edges with parameters
# Full AC power flow calculation (P, Q, V, δ) is complex and depends on load/gen.
# Here, we just label the line parameters (R, X).
edge_labels = {}
for u, v, data in G.edges(data=True):
    label = f"R={data['resistance']} Ω\nX={data['reactance']} Ω"
    # label = f"R={data['resistance']}\nX={data['reactance']}\nRating={data['rating']} MVA" # Example including rating
    edge_labels[(u, v)] = label

nx.draw_networkx_edges(
    G, pos, edge_color="gray", width=2, arrowstyle="->", arrowsize=20
)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=10)

# Annotate Ohm's Law and Kirchhoff's Laws
plt.text(
    0.5, 0.1,
    "Power Network\n"
    "Lines represented by Impedance Z = R + jX\n"
    "AC Power Flow depends on V magnitudes and angles",
    ha="center", va="center", fontsize=12, bbox=dict(facecolor="white", alpha=0.8)
)

plt.title("Power System Network Representation (AC Parameters)", fontsize=14)
plt.axis("off")
plt.tight_layout()

# Save the figure
# plt.savefig("power_system_flow.png", dpi=300, bbox_inches="tight")
plt.show()

Energy flow

In [ ]:
import plotly.graph_objects as go

# Define energy flows (in arbitrary units, e.g., GWh)
generation = 50
imports = 20
storage_discharge = 10
load = 60
exports = 5
storage_charging = 8
losses = 7

# Validate energy balance (generation + imports + discharge = load + exports + charging + losses)
assert (
    generation + imports + storage_discharge
    == load + exports + storage_charging + losses
), "Energy balance equation is not satisfied!"

# Sankey diagram setup
fig = go.Figure(go.Sankey(
    arrangement="snap",
    node={
        "label": [
            "Generation", "Imports", "Storage (Discharge)",  # Sources
            "Load", "Exports", "Storage (Charge)", "Losses",  # Sinks
            "Supply", "Demand"  # Aggregators
        ],
        "color": [
            "green", "blue", "orange",  # Sources
            "red", "purple", "cyan", "gray",  # Sinks
            "lightgreen", "lightcoral"  # Aggregators
        ],
    },
    link={
        "source": [
            0, 1, 2,  # Sources -> Supply
            7, 7, 7, 7  # Demand -> Sinks
        ],
        "target": [
            7, 7, 7,  # Sources -> Supply
            3, 4, 5, 6  # Demand -> Load, Exports, etc.
        ],
        "value": [
            generation, imports, storage_discharge,  # Flows into Supply
            load, exports, storage_charging, losses  # Flows out of Demand
        ],
        "color": [
            "rgba(0, 128, 0, 0.3)", "rgba(0, 0, 255, 0.3)", "rgba(255, 165, 0, 0.3)",  # Source colors
            "rgba(255, 0, 0, 0.3)", "rgba(128, 0, 128, 0.3)", "rgba(0, 255, 255, 0.3)", "rgba(128, 128, 128, 0.3)"  # Sink colors
        ],
    }
))

# Update layout
fig.update_layout(
    title="Generation + Imports + Storage Discharge = Load + Exports + Storage Charging + Losses<br>",
        #   "<sup></sup>",
    font=dict(size=10),
    height=600
)

# Save as HTML or display
# fig.write_html("energy_balance.html")
fig.show()

Energy Balance

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data: demand, wind, solar, backup, curtailment with added variability
np.random.seed(42) # for reproducibility
time = np.linspace(0, 24, 100)

# More realistic demand: base + peak + noise
base_demand = 80
peak_demand = 40 * np.sin(2 * np.pi * (time - 9) / 24) # Afternoon peak shifted
demand_noise = np.random.normal(0, 3, time.shape) # Add random fluctuations
demand = base_demand + np.maximum(0, peak_demand) + demand_noise

# More realistic wind: base profile + gustiness (noise)
wind_profile = 50 * (np.sin(2 * np.pi * (time - 6) / 24) + 0.8) / 1.8 # Base availability
wind_noise = np.random.normal(0, 5, time.shape) # Gustiness
wind = np.maximum(0, wind_profile + wind_noise)

# More realistic solar: profile based on sun angle + cloudiness (noise)
solar_profile = 60 * np.maximum(0, np.cos(2 * np.pi * (time - 12.5) / 24))**1.5 # Steeper rise/fall around noon
solar_noise = np.random.normal(0, 4, time.shape)
cloud_factor = np.maximum(0.2, 1 - 0.8 * np.random.rand(time.shape[0])**2) # Random cloud cover reducing output
solar = np.maximum(0, solar_profile * cloud_factor + solar_noise)

# Recalculate balance
generation = wind + solar
mismatch = demand - generation
backup = np.maximum(mismatch, 0)
curtailment = np.maximum(-mismatch, 0)

# Plotting the balance between demand and supply
plt.figure(figsize=(12, 7))
plt.plot(time, demand, label='Demand', color='black', linewidth=2)
plt.plot(time, wind, label='Wind Generation', color='deepskyblue', alpha=0.8)
plt.plot(time, solar, label='Solar Generation', color='orange', alpha=0.8)
plt.plot(time, generation, label='Total VRE Gen', color='green', linestyle='--', linewidth=1.5)

# Use fill_between for backup and curtailment based on mismatch
plt.fill_between(time, demand, generation, where=demand > generation, color='red', alpha=0.4, label='Energy Deficit (Backup Need)')
plt.fill_between(time, generation, demand, where=generation > demand, color='purple', alpha=0.4, label='Excess Energy (Curtailment/Storage)')

plt.title('Balancing Variable Demand and Renewable Generation (Illustrative)')
plt.xlabel('Time (hours)')
plt.ylabel('Power (MW)')
plt.legend(loc='upper left')
plt.grid(True, linestyle=':', alpha=0.7)
plt.xlim(0, 24)
plt.ylim(bottom=0)
plt.tight_layout()
plt.show()


Demand side management

In [ ]:
import numpy as np

# --- Regenerate Base Data for Consistent Comparison ---
np.random.seed(42) # for reproducibility
time = np.linspace(0, 24, 100)
time_step = time[1] - time[0] # Hours per time step

# Realistic demand: base + peak + noise
base_demand = 80
peak_demand_factor = 45 * np.sin(np.pi * (time - 8) / 12) # Peak centered around 2 PM (hour 14)
demand_noise = np.random.normal(0, 4, time.shape)
demand = base_demand + np.maximum(0, peak_demand_factor) + demand_noise
demand = np.maximum(demand, 20) # Min load

# Realistic wind: base profile + gustiness
wind_profile = 55 * (np.sin(2 * np.pi * (time - 6) / 24) + 0.8) / 1.8
wind_noise = np.random.normal(0, 6, time.shape)
wind = np.maximum(0, wind_profile + wind_noise)

# Realistic solar: profile + cloudiness
solar_profile = 70 * np.maximum(0, np.cos(2 * np.pi * (time - 12.5) / 24))**1.8
solar_noise = np.random.normal(0, 5, time.shape)
cloud_factor = np.maximum(0.3, 1 - 0.7 * np.random.rand(time.shape[0])**2)
solar = np.maximum(0, solar_profile * cloud_factor + solar_noise)

# Total renewable generation
generation = wind + solar

# --- Define DSM Strategies & Parameters ---
peak_threshold = 110 # kW, for shaving/shifting
efficiency_factor = 0.85 # 15% reduction
peak_indices = np.where((time >= 12) & (time <= 18) & (demand > peak_threshold))[0] # Peak ~noon-6pm
target_shift_indices = np.where((time >= 18) & (time <= 22))[0] # Target evening hours 6pm-10pm

# 1. Peak Shaving Profile
load_peak_shaving = np.minimum(demand, peak_threshold)

# 2. Load Shifting Profile (to Evening)
load_shifted_evening = demand.copy()
energy_to_shift = 0
if len(peak_indices) > 0:
    energy_above_threshold = (demand[peak_indices] - peak_threshold) * time_step
    energy_to_shift = np.sum(energy_above_threshold)
    load_shifted_evening[peak_indices] = peak_threshold # Cap the peak

if len(target_shift_indices) > 0 and energy_to_shift > 0:
    target_duration = len(target_shift_indices) * time_step
    power_increase_target = energy_to_shift / target_duration
    load_shifted_evening[target_shift_indices] += power_increase_target

# 3. Efficiency Improvement Profile
load_efficiency = demand * efficiency_factor

# --- Create 2x2 Subplots ---
fig, axs = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
fig.suptitle('Impact of DSM Strategies on Meeting Demand with Renewables', fontsize=16)

# Helper function to plot a scenario
def plot_scenario(ax, title, demand_profile, gen_profile, time_vector):
    ax.plot(time_vector, gen_profile, label='Renewable Gen (Solar+Wind)', color='green', alpha=0.7)
    ax.plot(time_vector, demand_profile, label='Demand Profile', color='black', linestyle='--')
    
    # Fill deficit (Demand > Generation)
    ax.fill_between(time_vector, demand_profile, gen_profile, where=demand_profile > gen_profile, 
                     color='red', alpha=0.4, label='Deficit (Need Backup/Storage)')
    # Fill surplus (Generation > Demand)
    ax.fill_between(time_vector, demand_profile, gen_profile, where=gen_profile >= demand_profile, 
                     color='lightgreen', alpha=0.4, label='Surplus (Potential Curtailment/Storage)')
    
    ax.set_title(title)
    ax.grid(True, linestyle=':', alpha=0.6)
    ax.set_ylim(bottom=0)
    ax.set_xlim(0, 24)

# Plot each scenario
plot_scenario(axs[0, 0], 'Baseline (No DSM)', demand, generation, time)
plot_scenario(axs[0, 1], f'Peak Shaving (Cap @ {peak_threshold}kW)', load_peak_shaving, generation, time)
plot_scenario(axs[1, 0], 'Load Shifting (Peak to Evening)', load_shifted_evening, generation, time)
plot_scenario(axs[1, 1], f'Efficiency Improvement ({int((1-efficiency_factor)*100)}%)', load_efficiency, generation, time)

# Add shared labels
for ax in axs[:, 0]:
    ax.set_ylabel('Power (kW)')
for ax in axs[1, :]:
    ax.set_xlabel('Time (hours)')

# Create a single legend for the figure
handles, labels = axs[0, 0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.95), ncol=4, fontsize=10)

plt.tight_layout(rect=[0, 0.03, 1, 0.93]) # Adjust layout to prevent overlap and make space for suptitle/legend
plt.show()


Frequenscy VS imbalanced grid

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Time axis (hours)
time = np.linspace(0, 24, 100)

# Simulate demand (typical daily profile)
demand = 50 + 30 * np.sin(2 * np.pi * time/24 - np.pi/2)

# Simulate supply scenarios
supply_balanced = demand  # Perfect match
supply_under = demand - 10  # Constant under-supply
supply_over = demand + 8  # Constant over-supply
supply_volatile = demand + 10 * np.random.randn(len(time))  # Unstable supply

# Calculate imbalance
imbalance_under = supply_under - demand
imbalance_over = supply_over - demand
imbalance_volatile = supply_volatile - demand

# Create figure
plt.figure(figsize=(10, 6))

# Plot demand and supply
plt.subplot(2, 1, 1)
plt.plot(time, demand, label='Demand', linewidth=2)
plt.plot(time, supply_under, '--', label='Under-Supply')
plt.plot(time, supply_over, '--', label='Over-Supply')
plt.plot(time, supply_volatile, '--', label='Volatile Supply')
plt.ylabel('Power (MW)')
plt.title('Grid Supply-Demand Imbalance')
plt.legend()
plt.grid(True)

# Plot frequency deviation (simplified)
plt.subplot(2, 1, 2)
plt.plot(time, 50 + 0.5*imbalance_under, label='Under-Supply')
plt.plot(time, 50 + 0.5*imbalance_over, label='Over-Supply')
plt.plot(time, 50 + 0.5*imbalance_volatile, label='Volatile Supply')
plt.axhline(50, color='black', linestyle='--', label='Nominal (50Hz)')
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (hours)')
plt.title('Grid Frequency Deviation')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('grid_imbalance.png', dpi=300)
plt.show()